In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
# hyperparameters
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_hidden = 32
n_head = 4
n_layer = 3
# ------------

torch.manual_seed(1337)

In [4]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [52]:
class RNNCell(nn.Module):
    # take input + hidden state at previous timestep and output new hidden state
    # this is a very simple RNN Cell:
    # concatenate xh and hprev --> linear layer --> ht
    def __init__(self):
        super().__init__()
        self.xh_to_ht = nn.Linear(n_embd + n_hidden, n_hidden)
    
    def forward(self, x, hprev):
        xh = torch.cat((x, hprev), 1)  # B, n_embd + n_hidden
        ht = F.tanh(self.xh_to_ht(xh)) # update state
        return ht

class GRUCell(nn.Module):
    # GRU implementation
    # https://arxiv.org/pdf/1406.1078.pdf
    def __init__(self):
        super().__init__()
        self.xh_to_r = nn.Linear(n_embd + n_hidden, n_hidden)
        self.xh_to_z = nn.Linear(n_embd + n_hidden, n_hidden)
        self.xh_to_hbar = nn.Linear(n_embd + n_hidden, n_hidden)
    
    def forward(self, xt, hprev):
        # concatenate x with previous hidden state
        xh = torch.cat((xt, hprev), 1) # B, n_embd + n_hidden

        # set some hidden state channels to zero 
        r = F.sigmoid(self.xh_to_r(xh)) # B, n_hidden
        hprev_reset = r * hprev

        # calculate the candidate new hidden state
        xh_reset = torch.cat((xt, hprev_reset), 1) # B, n_embd + n_hidden
        hbar = F.tanh(self.xh_to_hbar(xh_reset)) # B, n_hidden

        # calculate the switch gate to determine if a channel should be updated at all
        z = F.sigmoid(self.xh_to_z(xh)) # B, n_hidden

        # combine everything to form new hidden state
        ht = (1 - z) * hprev + z * hbar
        
        return ht

class RNN(nn.Module):
    def __init__(self, cell):
        super().__init__()
        self.wte = nn.Embedding(vocab_size,  n_embd)
        self.hidden_init = nn.Parameter(torch.zeros(1, n_hidden)) 
        if cell == 'rnn':
            self.cell = RNNCell()
        elif cell == 'gru':
            self.cell = GRUCell()
        self.lm_head = nn.Linear(n_hidden, vocab_size)
    
    def forward(self, idx, targets = None):
        b, t = idx.shape

        # embed input 
        emb = self.wte(idx) # B, T, n_embd

        # update hidden state --> the hidden state stores a vector of memory
        hprev = self.hidden_init.expand((b, -1)) # B, n_hidden

        # we need T hidden state updates
        hiddens = []
        for i in range(t):
            xt = emb[:, i, :] # B, C
            ht = self.cell(xt, hprev) # B, n_hidden
            hprev = ht
            hiddens.append(ht) # list containing T tensors with dim B, n_hidden

        hidden = torch.stack(hiddens, 1) # B, T, n_hidden
        logits = self.lm_head(hidden) # B, T, vocab_size     Why @ ht = y
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

In [7]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) * C **-0.5 # (B,T,C) @ (B,C,T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim = 1) # (B,T,T)
        # perform weighted aggregation of values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B,T,C)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) # projection back into residual pathway
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection layer going back into residual pathway
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer Block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: number of heads
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))   # pre-norm
        x = x + self.ffwd(self.ln2(x)) # pre-norm
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)    # tokenize 
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # positional information
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.lnf = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None): 
        B, T, = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(T, device = device)) # (T, C)
        x = tok_embd + pos_embd # (B,T,C)
        x = self.blocks(x)      # (B,T,C)  pass through transformer blocks
        x = self.lnf(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

In [71]:
#model = RNN('gru') # rnn for RNN cell, gru for GRU cell
#model = BigramLanguageModel()
#m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 0.4507, val loss 0.4475
step 500: train loss 0.3976, val loss 0.4023
step 1000: train loss 0.4100, val loss 0.4405
step 1500: train loss 0.4050, val loss 0.4395
step 2000: train loss 0.3805, val loss 0.3920
step 2500: train loss 0.3856, val loss 0.3800
step 3000: train loss 0.3831, val loss 0.3856
step 3500: train loss 0.3839, val loss 0.3765
step 4000: train loss 0.3916, val loss 0.3964
step 4500: train loss 0.3559, val loss 0.3488


In [56]:
@torch.no_grad()
def generate(model, idx, max_new_tokens = 100, temperature = 1.0):
    for _ in range(max_new_tokens):
        # crop token sequence so it's no longer than block size
        idx_cond = idx if idx.size(1) <= block_size else idx[:,-block_size:]

        # pass tokens through model
        logits, _ = model(idx_cond)

        # pluck out logits and divide by temperature to effect sampling confidence
        logits = logits[:, -1, :] / temperature

        # calculate probabilities
        probs = F.softmax(logits, dim = -1)

        # sample probabilities to get next token
        idx_next = torch.multinomial(probs, num_samples=1)

        # concatenate new token to end of token sequence
        idx = torch.cat((idx, idx_next), dim = 1)

    return idx

In [60]:
# RNN Cell
X_init = torch.zeros(1, 1, dtype=torch.long)
print(decode(generate(m, X_init, max_new_tokens=300)[0].tolist()))



Srrw, Telethy pe anlalm,
Gics?
The ten lowt mith meall, whou ge themews so gired'Th tore wothte vidot thot to IUbugro-dond nowaworeake gily mitern,
And hor he gath or that th the wicks I And weon, pers, agst todUSINHOPE:
Oel Nupome d-tels, seand, ssawa pent
Hellire let lyou momaurt.
Nort I aid my,




In [64]:
# GRU Cell
X_init = torch.zeros(1, 1, dtype=torch.long)
print(decode(generate(m, X_init, max_new_tokens=300)[0].tolist()))


Cow grore an-wemser and yovere, marmair me faach nomer, tisllive;
O dot seny'd nellast, I shade
Thebllly will mewuving to do my bay a laich so, Shou prranen
Whous you gilice you:

Mem.

MOLENTENEOUE:
When confer and ku
pich live me, grey: lown to dist if yeap gthavey the lors ir ere to and or hich o


In [73]:
# Transformer :)
X_init = torch.zeros(1, 1, dtype=torch.long)
print(decode(generate(m, X_init, max_new_tokens=300)[0].tolist()))


IIIZCOSO:
KOS
IKAsngas d'se cent ame dnld:
AwcastM Iows

MRIR:
LAnd ab.

OThCORIH
CII:
PeEvl
MOUCOECEI:

And mO Iodbwei.

NIW
LAOLILOSUB:
IOC:
Pure whsin dw?

NEWII:

RAMUI COOTALEOLII
THO CILNAREDEHEHIUCHA:

COASSIO:

OSiTe,
c'aw show weimey I and wOw Ihe, mecwem-ids
L
Cis: dor dotle mond sise me,

